# Машинное обучение. Примеры и разборы. Лабораторная работа 1.

## Пример первый. Поиск закономерности.

Пример взят [отсюда](https://neurohive.io/ru/tutorial/prostaja-nejronnaja-set-python/).

Данные для обучения: четыре примера с результатами.

Результат: вероятность того, что следующая последовательность будет соответствовать правилам выбора (будет из той же серии последовательностей, и т.п.).

Сеть имеет ошибку: при вводе трёх нулей все веса равны нулю, и итоговый выход всегда равен 0.5

In [8]:
from numpy import exp, array, random, dot

# вводные данные: последовательности
training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])

# вводные данные: результат проверки того, построена эта последовательность по заданным правилам или нет
training_set_outputs = array([[0, 1, 1, 0]]).T

# инициализация генератора случайных чисел - для генерации случайных весов нейронов
random.seed(1)

# вес может быть от -1 до 1
synaptic_weights = 2 * random.random((3, 1)) - 1 

# количество итераций повтора и подбора весов нейронной сети таким образом, 
# чтобы её выход соответствовал training_set_outputs
for iteration in range(10000):
    output = 1 / (1 + exp(-(dot(training_set_inputs, synaptic_weights))))
    synaptic_weights += dot(training_set_inputs.T, (training_set_outputs - output) * output * (1 - output))

In [10]:
# теперь ищем вероятность того, что [1, 0, 0] построена по заданным правилам
print(1 / (1 + exp(-(dot(array([0, 0, 0]), synaptic_weights)))))

[0.5]


In [3]:
print(1 / (1 + exp(-(dot(array(list(map(int, input("Введите три числа последовательности в одну строку: ")
                                        .split()))), synaptic_weights)))))

Введите три числа последовательности в одну строку: 0 1 0
[0.44822538]


In [4]:
from numpy import exp, array, random, dot

class SimpleModel:
    def __init__(self, training_set_inputs, training_set_outputs):
        self.training_set_inputs = training_set_inputs
        self.training_set_outputs = training_set_outputs
        random.seed(1)
        self.synaptic_weights = 2 * random.random((len(training_set_inputs[0]), 1)) - 1
        print('Нейронная сеть инициализирована.')
        
    def sigmoid(self, set_inputs):
        return 1 / (1 + exp(-(dot(set_inputs, self.synaptic_weights))))
        
    def learn(self, iterations):
        for iteration in range(iterations):
            output = self.sigmoid(self.training_set_inputs)
            self.synaptic_weights += dot(self.training_set_inputs.T, 
                                    (self.training_set_outputs - output) * output * (1 - output))
        print(f'Нейронная сеть обучена на {iterations} итерациях.')

In [5]:
# вводные данные: последовательности
training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])

# вводные данные: результат проверки того, построена эта последовательность по заданным правилам или нет
training_set_outputs = array([[0, 1, 1, 0]]).T

model = SimpleModel(training_set_inputs, training_set_outputs)
model.learn(10000)
print(model.sigmoid([1, 0, 0]))

Нейронная сеть инициализирована.
Нейронная сеть обучена на 10000 итерациях.
[0.99993704]


## Пример второй. Два слоя нейронной сети и алгоритм обратного распространения ошибок на том же примере.

Пример взят [отсюда](https://habr.com/ru/post/271563/).

Здесь решается проблема с вводом трёх нулей: вероятность становится меньше половины.

In [11]:
X = array([ [0,0,1],[1,1,1],[1,0,1],[0,1,1] ])
y = array([[0,1,1,0]]).T
syn0 = 2*random.random((3,4)) - 1 # слой первый
syn1 = 2*random.random((4,1)) - 1 # слой второй
for j in range(60000):
    l1 = 1/(1+exp(-(dot(X,syn0))))
    l2 = 1/(1+exp(-(dot(l1,syn1))))
    l2_delta = (y - l2)*(l2*(1-l2))
    l1_delta = l2_delta.dot(syn1.T) * (l1 * (1-l1))
    syn1 += l1.T.dot(l2_delta)
    syn0 += X.T.dot(l1_delta)

In [12]:
print(1/(1+exp(-(dot(1/(1+exp(-(dot([0, 0, 0],syn0)))),syn1)))))

[0.12648418]


## Пример третий. Определение оптимального чая.

> Определить оптимальный чай для человека. <br><br>
Входные параметры: <br>
• Температура на улице (-100 – 100°С) <br>
• Температура в доме (-50 – 50°С) :) <br> 
• Время суток (00:00 – 23:59) <br>
• Месяц, число, день недели (пн, вт, ср, чт, пт, сб, вс) <br>
• Темперамент человека (холерик, флегматик,сангвиник, меланхолик) <br>
• Настроение человека (радость в единицах от 0 до 10) <br><br>
Ответ нейронной сети: <br>
• Тип чая (черный, зеленый, красный, ...) <br>
• Температура (0–120°С) <br>
• Обьем (20–500мл) <br><br>
Для обучения использовать датасет с открытых источников, а также эксперименты над собой и близкими.

**Пример взят из Telegram, мы слегка его упростили:**

• Температура на улице (-20 – 20°С) <br>
• Температура в доме (-20 – 25°С) <br>
• Время суток (00 – 23) <br>
• День недели (0 - чётный, 1 - нечётный) <br>
• Темперамент человека (холерик, флегматик, сангвиник, меланхолик) <br>
• Настроение человека (радость в единицах от 0 до 10) <br><br>
Ответ нейронной сети: <br>
• Тип чая (черный, зеленый, красный) <br>
• Температура (0 – 100°С) <br>

**Затем нам понадобилось сменить диапазон у данных (все данные должны равномерно распределить диапазон от 0 до 1).**

In [1]:
# функция преобразования диапазона
def rearrange(array, old_min, old_max, new_min, new_max):
    for index in range(len(array)): 
        array[index] = ( (array[index] - old_min) / (old_max - old_min) ) * (new_max - new_min) + new_min 
    return array

Затем мы загрузили данные в таблицу в редактор таблиц и сохранили как "Текст CSV". Данные приведены ниже:

In [6]:
from numpy import array

inputs = array([[0.2  , 0.84 , 0.52 , 0.   , 0.66 , 0.6  ],
       [0.5  , 0.84 , 0.43 , 1.   , 0.33 , 0.   ],
       [0.   , 0.71 , 0.47 , 0.   , 0.   , 0.   ],
       [0.925, 0.82 , 0.52 , 1.   , 1.   , 0.1  ],
       [0.975, 0.97 , 0.52 , 0.   , 1.   , 0.4  ],
       [0.225, 0.66 , 0.78 , 1.   , 0.66 , 0.6  ],
       [0.   , 0.64 , 0.6  , 0.   , 0.66 , 0.   ],
       [0.8  , 0.35 , 0.56 , 1.   , 0.33 , 0.7  ],
       [0.2  , 0.4  , 0.69 , 0.   , 0.   , 0.2  ]])

outputs = array([[1. , 0.5],
       [1. , 0.8],
       [0. , 0.7],
       [1. , 0.5],
       [1. , 0.5],
       [0. , 0.9],
       [0. , 0.8],
       [0.5, 0.9],
       [0. , 0.4]])

Создадим класс модели:

In [7]:
import pandas as pd
import numpy as np
import time

class TeaModel:
    # статичные методы
    def rearrangeSet(dataset):
        rearranged_dataset = [np.array() * len(dataset)]
        for array_index in len(dataset):
            print(f'Преобразование датасета {array_index}.')
            for column in len(dataset[array_index][0]):
                old_min, old_max = map(float, 
                                       input(f'Введите диапазоны данных {column} ' +
                                             'столбца (минимум и максимум) через пробел: ')
                                       .split())
                rearranged_dataset[array_index].append(
                    TeaModel.rearrange(
                        dataset[array_index].T[column], 
                        old_min, 
                        old_max))
            rearranged_dataset[array_index] = rearranged_dataset[array_index].T
        return rearranged_dataset
    
    def rearrange(array, old_min, old_max, new_min = 0, new_max = 1):
        for index in range(len(array)): 
            array[index] = ((array[index] - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min 
        return array
    
    # основной API
    def __init__(self, 
                 filename, 
                 number_of_expected_results, 
                 isRearranged = True, # данные уже преобразованы в диапазоны от 0 до 1?
                 isTrained = False    # нужно ли обучать модель, или пользователь подгрузит данные сам?
                ):
        print('Модель подбора идеального чая. Авторы: Шамиль Гаджиев и Клем Титов.')
        if isTrained:
            print('Прежде чем использовать модель, переопределите данные (layer1 и layer2, ' +
                  'или используйте метод setLayers(layer1, layer2)).')
            return
        dataset = pd.read_csv(filename).rename_axis('ID').values
        self.training_inputs = dataset.T[ : - number_of_expected_results ].T
        self.training_outputs = dataset.T[ - number_of_expected_results : ].T
        if not isRearranged:
            self.training_inputs, self.training_outputs = TeaModel.rearrangeSet([self.training_inputs,
                                                                                 self.training_outputs])
        if input('Вывести API? [n/y] ') == 'y':
            print('\tМетоды:')
            print('\t\ttrain(epochs, logEvery) - обучает нейронную сеть с заданным числом эпох')
            print('\t\tgetResult(inputs_array, isRearranged) - получает вывод нейросети на входные данные')
            print('\t\timode() - интерактивный режим ввода')
            print('\t\tsetLayers(layer1, layer2) - загружает два обученных слоя нейросети.')
        
    def train(self, epochs = 10000, logEvery = -1):
        print('Обучение сети.')
        np.random.seed(int(time.time()))
        self.layer1 = 2 * np.random.random((len(self.training_inputs[0]),len(self.training_inputs))) - 1
        self.layer2 = 2 * np.random.random((len(self.training_outputs), len(self.training_outputs[0]))) - 1
        for epoch in range(epochs):
            l1 = 1 / (1 + np.exp(-(np.dot(self.training_inputs, self.layer1))))
            l2 = 1 / (1 + np.exp(-(np.dot(l1, self.layer2))))
            l2_delta = (self.training_outputs - l2) * (l2 * (1 - l2))
            l1_delta = l2_delta.dot(self.layer2.T) * (l1 * (1 - l1))
            self.layer2 += l1.T.dot(l2_delta)
            self.layer1 += self.training_inputs.T.dot(l1_delta)
            if (epoch + 1) % logEvery == 0:
                print(f'Эпоха {epoch}')
                print('l1: ')
                print(l1)
                print()
                print('l2: ')
                print(l2)
                print()
                print('l1_delta: ')
                print(l1_delta)
                print()
                print('l2_delta: ')
                print(l2_delta)
                
    def getResult(self, inputs_array, isRearranged = True):
        if not isRearranged:
            inputs_array = TeaModel.rearrangeSet([inputs_array])[0]
        return (1 / 
                (1 + 
                 np.exp(-(np.dot(1 / 
                                 (1 + 
                                  np.exp(-(np.dot(inputs_array, 
                                                  self.layer1)))), 
                                 self.layer2)))))
    
    def imode():
        pass
    
    def setLayers(self, layer1, layer2):
        self.layer1 = layer1
        self.layer2 = layer2

И всё, начинаем использовать.

In [8]:
model = TeaModel('Данные для чая.csv', 2)

Модель подбора идеального чая. Авторы: Шамиль Гаджиев и Клем Титов.
Вывести API? [n/y] y
	Методы:
		train(epochs, logEvery) - обучает нейронную сеть с заданным числом эпох
		getResult(inputs_array, isRearranged) - получает вывод нейросети на входные данные
		imode() - интерактивный режим ввода
		setLayers(layer1, layer2) - загружает два обученных слоя нейросети.


In [9]:
model.train(logEvery = 10000)

Обучение сети.
Эпоха 9999
l1: 
[[0.26801273 0.19022116 0.84286507 0.61516936 0.13428518 0.68356973
  0.87456929 0.69551665 0.03790894]
 [0.25949335 0.18726783 0.98307269 0.483999   0.0507261  0.80872159
  0.73595749 0.73809897 0.19598661]
 [0.49973092 0.65043311 0.61975599 0.58424043 0.58081011 0.37374723
  0.41750774 0.4555598  0.08015834]
 [0.19244913 0.05834132 0.99721829 0.39351857 0.00738552 0.95003014
  0.95787627 0.86987292 0.0613195 ]
 [0.0766569  0.01761934 0.98261197 0.70570133 0.01858958 0.95151018
  0.98841753 0.92261545 0.00432712]
 [0.70093131 0.56687275 0.94705982 0.29131986 0.11375955 0.49387866
  0.8194662  0.29855289 0.51599009]
 [0.66688961 0.77187923 0.83375745 0.34110071 0.31447212 0.49748453
  0.5078533  0.48307522 0.03733066]
 [0.44782643 0.11082381 0.93968179 0.50320217 0.09699243 0.69474041
  0.97757513 0.38034134 0.77085245]
 [0.78320694 0.7904409  0.4378615  0.49803366 0.78488004 0.21928583
  0.7151384  0.16391072 0.240635  ]]

l2: 
[[9.92404061e-01 4.9993186

In [10]:
print(model.getResult([0.9, 0.66, 0.478, 0.0, 1.0, 0.4]))

[0.99988464 0.47309816]
